# Watershedアルゴリズム

このノートブックでは、watershedアルゴリズムについて学びます。

## 目次
1. [距離変換](#距離変換)
2. [マーカー](#マーカー)
3. [分水嶺](#分水嶺)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## 距離変換

In [ ]:
# 距離変換（Distance Transform）
# 各ピクセルから最も近い背景ピクセルまでの距離を計算

# サンプル画像の作成（複数のオブジェクト）
img = np.zeros((200, 200), dtype=np.uint8)
img[30:70, 30:70] = 255  # オブジェクト1
img[100:140, 100:140] = 255  # オブジェクト2
img[50:90, 130:170] = 255  # オブジェクト3

# 距離変換
dist_transform = cv2.distanceTransform(img, cv2.DIST_L2, 5)

# 正規化（可視化のため）
dist_normalized = cv2.normalize(dist_transform, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# 結果の表示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('元の画像（二値化）')
axes[0].axis('off')

axes[1].imshow(dist_transform, cmap='hot')
axes[1].set_title('距離変換')
axes[1].axis('off')

axes[2].imshow(dist_normalized, cmap='gray')
axes[2].set_title('距離変換（正規化）')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("距離変換:")
print("- 各ピクセルから最も近い背景までの距離を計算")
print("- オブジェクトの中心ほど大きな値を持つ")
print("- Watershedアルゴリズムの前処理として使用")

## マーカー

In [ ]:
# マーカーの作成
# Watershedアルゴリズムで使用するシード（マーカー）を作成

# 距離変換からマーカーを作成
# 距離変換の局所最大値を検出
_, markers = cv2.connectedComponents(np.uint8(dist_transform > 0.5 * dist_transform.max()))

# より正確なマーカー作成
# 距離変換のピークを検出
_, sure_fg = cv2.threshold(dist_normalized, 0.7 * dist_normalized.max(), 255, 0)

# 背景領域の確定
sure_bg = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations=3)

# 不明確な領域
unknown = cv2.subtract(sure_bg, np.uint8(sure_fg))

# マーカーの作成
_, markers = cv2.connectedComponents(np.uint8(sure_fg))
markers = markers + 1
markers[unknown == 255] = 0

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(dist_normalized, cmap='gray')
axes[0, 1].set_title('距離変換')
axes[0, 1].axis('off')

axes[0, 2].imshow(sure_fg, cmap='gray')
axes[0, 2].set_title('確実な前景')
axes[0, 2].axis('off')

axes[1, 0].imshow(sure_bg, cmap='gray')
axes[1, 0].set_title('確実な背景')
axes[1, 0].axis('off')

axes[1, 1].imshow(unknown, cmap='gray')
axes[1, 1].set_title('不明確な領域')
axes[1, 1].axis('off')

axes[1, 2].imshow(markers, cmap='nipy_spectral')
axes[1, 2].set_title(f'マーカー ({markers.max()}個の領域)')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print(f"検出されたオブジェクト数: {markers.max()}")
print("マーカー作成のステップ:")
print("1. 距離変換から確実な前景を決定")
print("2. 背景を拡張して確実な背景を決定")
print("3. 不明確な領域を識別")
print("4. 各オブジェクトに異なるラベルを割り当て")

## 分水嶺

In [ ]:
# Watershedアルゴリズム
# マーカーを基に画像をセグメント化

# カラー画像に変換（Watershedは3チャンネルを要求）
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# Watershedアルゴリズムの適用
markers = cv2.watershed(img_color, markers)

# 結果の可視化
img_result = img_color.copy()
img_result[markers == -1] = [0, 255, 0]  # 境界を緑色で表示

# 各領域に異なる色を割り当て
colored_markers = np.zeros_like(img_color)
for i in range(1, markers.max() + 1):
    mask = (markers == i)
    color = np.random.randint(0, 256, 3)
    colored_markers[mask] = color

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(markers, cmap='nipy_spectral')
axes[0, 1].set_title('Watershed結果（マーカー）')
axes[0, 1].axis('off')

axes[1, 0].imshow(img_result)
axes[1, 0].set_title('境界表示（緑色）')
axes[1, 0].axis('off')

axes[1, 1].imshow(colored_markers)
axes[1, 1].set_title('色分けされた領域')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"セグメント化された領域数: {markers.max()}")
print("Watershedアルゴリズム:")
print("- マーカーから開始して領域を成長")
print("- 境界（-1）で領域を分離")
print("- 接触しているオブジェクトを分離するのに有効")